Implementing the "hack" described in the paper  "Magpie: Alignment Data Synthesis from Scratch by Prompting Aligned LLMs with Nothing" (https://arxiv.org/abs/2406.08464) to generate an instruction dataset with the format:

{"instruction": What type of cloud is typically associated with thunderstorms?,<br>
"output": The type of cloud typically associated with thunderstorms is cumulonimbus.}

LLama 8B- Instruct model is used


Evaluate: Quality of instructions, Difficulty of instructions, Instruction SImilarity

In [1]:
#importing libraries
from tqdm import tqdm
import json

In [2]:
#ollama run llama3
#Verifying if Ollama 3 is running
import psutil
 
def check_if_running(process_name):
    running = False
    for proc in psutil.process_iter(["name"]):
        if process_name in proc.info["name"]:
            running = True
            break
    return running
 
ollama_running = check_if_running("ollama")
 
if not ollama_running:
    raise RuntimeError("Ollama not running. Launch ollama before proceeding.")
print("Ollama running:", check_if_running("ollama"))

Ollama running: True


In [12]:
#Function to interact with Ollama via REST API

import urllib.request
 
def query_model(prompt, model="llama3", url="http://localhost:11434/api/chat",role="user"):
    data = {
        "model": model,
        "seed": 123,        
        "temperature": 1.,
        "top_p":1,
        "messages": [
            {"role": role, "content": prompt}
        ]
    }
 
    payload = json.dumps(data).encode("utf-8")
    request = urllib.request.Request(url, data=payload, method="POST")
    request.add_header("Content-Type", "application/json")
 
    response_data = ""
    with urllib.request.urlopen(request) as response:
        while True:
            line = response.readline().decode("utf-8")
            if not line:
                break
            response_json = json.loads(line)
            response_data += response_json["message"]["content"]
 
    return response_data

In [13]:
result = query_model("What do Llamas eat?")
print(result)

Llamas are herbivores, which means they primarily eat plants and plant-based foods. Their diet typically consists of:

1. Grasses: Llamas love to graze on various types of grasses, including tall grasses, bunchgrasses, and wheat.
2. Leaves: They enjoy munching on leaves from trees, shrubs, and bushes, like willow, cottonwood, and alder.
3. Plants: Llamas eat a variety of plants, such as clover, alfalfa, and other legumes.
4. Fruits and veggies: In the wild or in captivity, llamas might also eat fruits and vegetables like apples, carrots, and sweet potatoes.
5. Hay: A staple in many llama diets is hay, which can be timothy grass, alfalfa, or a combination of both.

In terms of specific nutrients, llamas need:

* High-quality fiber (like hay) for digestive health
* Adequate protein from plants like legumes and grains
* Minerals like calcium, phosphorus, and potassium

In the wild, llamas roam in mountainous regions at high elevations, where they feed on whatever is available. In captivit

In [15]:
#Extract the instructions using the prompt template proposed in the paper
query_template = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>"

result = query_model(query_template, role="assistant")
instruction =  [instruction for instruction in result.split("\n") if instruction][0]
print(instruction)

What are some common myths about mental health?


In [16]:
#Generate the response
response=query_model(instruction, role="user")
print(response)

Unfortunately, there are many myths and misconceptions surrounding mental health that can perpetuate stigma, delay help-seeking, or hinder effective treatment. Here are some common myths about mental health:

1. **Myth:** Mental illness is a personal weakness or flaw.
**Reality:** Mental illnesses are medical conditions that affect the brain and can be caused by a combination of genetic, environmental, and lifestyle factors.

2. **Myth:** Only crazy people get mentally ill.
**Reality:** Mental illnesses can affect anyone, regardless of their background, education level, or occupation. They are more common than you think!

3. **Myth:** Therapy is for "losers" who can't cope with life.
**Reality:** Therapy (counseling) is a valuable tool for personal growth, stress management, and overcoming challenges. Many people seek therapy to improve their mental well-being.

4. **Myth:** You just need to "toughen up" or "snap out of it" if you're feeling down.
**Reality:** Mental health issues like

In [17]:
#Generate full dataset
size=10
dataset=[]

for i in tqdm(range(size)):
    result = query_model(query, role="assistant")
    instruction =  [instruction for instruction in result.split("\n") if instruction][0]
    response=query_model(instruction, role="user")
    dataset.append({'instruction':instruction, 'response':response})

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]


NameError: name 'query' is not defined

In [9]:
for entry in dataset:
    print(entry['instruction'],"\n",entry['response'])

NameError: name 'dataset' is not defined

In [52]:
with open("instruction-data-llama3-7b.json", "w") as file:
    json.dump(dataset, file, indent=4)

"I'm trying to determine the best way to organize my digital life. I have a ton of files and documents that are scattered across multiple devices, clouds, and platforms. I'd like to get everything in one place and make it easy to find what you need when you need it."

In [ ]:
!cat instruction-data-llama3-7b.json